# Decision Trees

### 0) Sample Data

In [34]:
import pandas as pd
data = {
    'color' : ['Purple', 'Purple', 'Yellow', 'Green', 'Green'],
    'size'  : [1       , 1.5     , 3       , 3      , 3      ],
    'fruit' : ['Grape' , 'Grape' , 'Lemon' , 'Apple', 'Lemon']
}


data = {
    'color' : ['Green', 'Yellow', 'Red'  , 'Red'  , 'Yellow'],
    'size'  : [3      , 3       , 1      , 1      , 3       ],
    'fruit' : ['Apple', 'Apple' , 'Grape', 'Grape', 'Lemon' ]
}

df = pd.DataFrame(data=data)
df.head()

,color,fruit,size
0,Green,Apple,3
1,Yellow,Apple,3
2,Red,Grape,1
3,Red,Grape,1
4,Yellow,Lemon,3


### 1) Split the Data
Trees work by "asking questions" and spliting the data accordingly.
The objective is to have a node with the purest data in it

In [35]:
class Condition:
    """
    A Condition is used to partition a dataset.
     
    This class just records a column name (e.g., Color) and a
    column value (e.g., Green)
    
    Numerical questions are always >=
    """
    
    def __init__(self, column_name, value):
        self.column_name = column_name
        self.value = value

    def __str__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if isinstance(self.value, int) or isinstance(self.value, float):
            condition = ">="
        return "Is %s %s %s?" % (self.column_name, condition, str(self.value))

In [36]:
def partition(df, condition):
    
    # if question is numeric
    if isinstance(condition.value, int) or isinstance(condition.value, float):
        true_branch  = df[df[condition.column_name] >= condition.value]
        false_branch = df[df[condition.column_name] <  condition.value]
    else:
        true_branch  = df[df[condition.column_name] == condition.value]
        false_branch = df[df[condition.column_name] != condition.value]

    return true_branch, false_branch
    

In [37]:
### Tests
cond = Condition('color', 'Green')
print(cond)

Is color == Green?


In [38]:
green, not_green = partition(df, cond)

In [39]:
green

,color,fruit,size
0,Green,Apple,3


In [40]:
not_green

,color,fruit,size
1,Yellow,Apple,3
2,Red,Grape,1
3,Red,Grape,1
4,Yellow,Lemon,3


### 2) Calculate impurity of node and information gain on split

In [41]:
classes = df['fruit'].unique()
classes

array(['Apple', 'Grape', 'Lemon'], dtype=object)

In [42]:
df[df['fruit'] == 'Grape'].shape

(2, 3)

In [43]:
def gini(df, label_column_name):
    classes = df[label_column_name].unique()
    impurity = 1
    for label in classes:
        prob_of_lbl = df[df[label_column_name] == label].shape[0] / float(df.shape[0])
        impurity -= prob_of_lbl**2
    return impurity
    

In [44]:
gini(df, 'fruit')

0.6399999999999999

In [45]:
def info_gain(left, right, parent, label_column_name):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(left.shape[0]) / (left.shape[0] + right.shape[0])
    return gini(parent, label_column_name) - p * gini(left, label_column_name) - (1 - p) * gini(right, label_column_name)

In [46]:
# How much information do we gain by partioning on 'Green'?
true_rows, false_rows = partition(df, Condition('color', 'Green'))
info_gain(true_rows, false_rows, df, 'fruit')

0.1399999999999999

In [31]:
def find_best_split(df):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(df)
    n_features = df.shape[1] - 1  # number of columns - label

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

TypeError: object of type 'int' has no len()

In [49]:
for i in range(0, 150, 5):
    print("i = %s, P = %s" % (i, (100*100)/((25+i)*(25+i))*i))

i = 0, P = 0.0
i = 5, P = 55.55555555555556
i = 10, P = 81.63265306122449
i = 15, P = 93.75
i = 20, P = 98.76543209876543
i = 25, P = 100.0
i = 30, P = 99.17355371900827
i = 35, P = 97.22222222222221
i = 40, P = 94.67455621301775
i = 45, P = 91.83673469387756
i = 50, P = 88.88888888888889
i = 55, P = 85.9375
i = 60, P = 83.04498269896193
i = 65, P = 80.24691358024691
i = 70, P = 77.5623268698061
i = 75, P = 75.0
i = 80, P = 72.56235827664399
i = 85, P = 70.24793388429752
i = 90, P = 68.05293005671078
i = 95, P = 65.97222222222221
i = 100, P = 64.0
i = 105, P = 62.130177514792905
i = 110, P = 60.35665294924554
i = 115, P = 58.673469387755105
i = 120, P = 57.07491082045184
i = 125, P = 55.55555555555555
i = 130, P = 54.11030176899063
i = 135, P = 52.734375
i = 140, P = 51.423324150596876
i = 145, P = 50.173010380622834
